## An optimization test bed.

First, I'll define the usual functions.

In [1]:
def is_pos_def(X):
    '''Check whether a matrix X is pos definite.
    Returns True or False, depending on X.
    '''
    return np.all(np.linalg.eigvals(X) > 0)

def batch_data(data, batch_size):
    n = data.shape[0]
    p = data.shape[1]
    if n % batch_size != 0:
        n = (n // batch_size) * batch_size
    ind = np.arange(n)
    np.random.shuffle(ind)
    n_batches = n // batch_size
    data = data[ind].reshape(batch_size, p, n_batches)
    return(data, n_batches)

def sghmc(gradU, eta, niter, alpha, theta_0, V_hat, dat, batch_size):
    '''Define SGHMC as described in the paper
    Tianqi Chen, Emily B. Fox, Carlos Guestrin 
    Stochastic Gradient Hamiltonian Monte Carlo 
    ICML 2014.

    The inputs are:
    gradU = gradient of U
    eta = eps^2 M^(-1)
    niter = number of samples to generate
    alpha = eps M^(-1) C
    theta_0 = initial val of parameter(s) to be sampled
    V_hat = estimated covariance matrix of stoch grad noise
    See paper for more details

    The return is:
    A np.array of positions of theta.'''

    ### Initialization and checks ###
    # get dimension of the thing you're sampling
    p = len(theta_0)
    # set up matrix of 0s to hold samples
    n = dat.shape[0]
    theta_samps = np.zeros((p, niter*(n // batch_size)))
    # fix beta_hat as described on pg. 6 of paper
    beta_hat = 0.5 * V_hat @ eta
    # We're sampling from a N(0, 2(alpha - beta_hat) @ eta)
    # so this must be a positive definite matrix
    Sigma = 2 * (alpha - beta_hat) @ eta
    if not is_pos_def( Sigma ): 
        print("Error: (alpha - beta_hat) eta not pos def")
        return
    # Need batch size to be <= the amount of data
    if (batch_size > dat.shape[0]): 
        print("Error: batch_size must be <= number of data points")
        return

    # initialize nu and theta 
    nu = np.random.multivariate_normal(np.zeros(p), eta).reshape(p,-1)
    theta = theta_0
    
    # loop through algorithm to get niter samples
    it = 0
    for i in range(niter):
        dat_resh, nbatches = batch_data(dat, batch_size)
        
        # Resample momentum every epoch
        nu = np.random.multivariate_normal(np.zeros(p), eta).reshape(p,-1)
        
        for batch in range(nbatches):
            gradU_batch = gradU(theta, dat_resh[:,:,batch], n, batch_size).reshape(p,-1)
            nu = nu - eta @ gradU_batch - alpha @ nu + \
                 np.random.multivariate_normal(np.zeros(p), Sigma).reshape(p, -1)
            theta = theta + nu
            theta_samps[:,it] = theta.reshape(-1,p)
            it = it + 1
        
    return theta_samps

Now import some optimization things.

In [2]:
import numba
from numba import jit
import time

For now, just look at optimization for the mixture of normals problem. I don't care about performance of the sampler itself right now, so I turn down the size of n and niter just to make it go relatively quickly in my tests.

In [3]:
import matplotlib.pyplot as plt
import autograd.numpy as np
from autograd import jacobian
import seaborn as sns

## Example #1:
## Sampling from a mixture of normals in 1-D
## SAMPLING MODEL: x ~ 0.5 * N(mu1, 1) + 0.5 * N(mu2, 1)
## PRIORS: p(mu1) = p(mu2) = N(0,10)

def log_prior(theta):
    return(-(1/(2*10))*theta.T@theta)
      
def log_lik(theta, x):
    return(np.log(0.5 * np.exp(-0.5*(theta[0]-x)**2) + 0.5* np.exp(-0.5*(theta[1]-x)**2)))

def U(theta, x, n, batch_size):
    return(-log_prior(theta) - (n/batch_size)*sum(log_lik(theta, x)))
       
# Automatic differentiation to get the gradient
gradU = jacobian(U, argnum=0)

# Set random seed
np.random.seed(1234)
# Set up the data
p = 2 #dimension of theta
theta = np.array([-3.0, 3.0]).reshape(p,-1)
n = 100 # smaller for test
x = np.array([np.random.normal(theta[0], 1, (n,1)),
              np.random.normal(theta[1], 1, (n,1))]).reshape(-1,1)

## Initialize parameters and sample 

# Initialize mean parameters
#theta_0 = np.random.normal(size=(p,1))
theta_0 = theta # initialize at "true" value for testing

# Initialize tuning parameters:
# learning rate
eta = 0.01/n * np.eye(p)
# Friction rate
alpha = 0.1 * np.eye(p)

# Arbitrary guess at covariance of noise from mini-batching the data
V = np.eye(p)*1
niter = 50
batch_size=20 # make this smallish

# Don't actually run sampling algorithm here
# samps = sghmc(gradU, eta, niter, alpha, theta_0, V, x, batch_size)

Try cleaning up the multivariate normal sampling. Pre-calculate the Cholesky decompositions, and use the Cholesky-based sampling method.

In [4]:
def sghmc_cleaned(gradU, eta, niter, alpha, theta_0, V_hat, dat, batch_size):
    '''Define SGHMC as described in the paper
    Tianqi Chen, Emily B. Fox, Carlos Guestrin 
    Stochastic Gradient Hamiltonian Monte Carlo 
    ICML 2014.

    The inputs are:
    gradU = gradient of U
    eta = eps^2 M^(-1)
    niter = number of samples to generate
    alpha = eps M^(-1) C
    theta_0 = initial val of parameter(s) to be sampled
    V_hat = estimated covariance matrix of stoch grad noise
    See paper for more details

    The return is:
    A np.array of positions of theta.'''

    ### Initialization and checks ###
    # get dimension of the thing you're sampling
    p = len(theta_0)
    # set up matrix of 0s to hold samples
    n = dat.shape[0]
    theta_samps = np.zeros((p, niter*(n // batch_size)))
    # fix beta_hat as described on pg. 6 of paper
    beta_hat = 0.5 * V_hat @ eta
    # We're sampling from a N(0, 2(alpha - beta_hat) @ eta)
    # so this must be a positive definite matrix
    Sigma = 2 * (alpha - beta_hat) @ eta
    Sig_chol = np.linalg.cholesky(Sigma)
    if not is_pos_def( Sigma ): 
        print("Error: (alpha - beta_hat) eta not pos def")
        return
    # Need batch size to be <= the amount of data
    if (batch_size > n): 
        print("Error: batch_size must be <= number of data points")
        return

    # initialize nu and theta 
    nu = np.random.multivariate_normal(np.zeros(p), eta).reshape(p,-1)
    theta = theta_0
    
    # set up for Chol decomp for MV normal sampling of nu every epoch
    eta_chol = np.linalg.cholesky(eta)
    
    # loop through algorithm to get niter samples
    it = 0
    for i in range(niter):
        dat_resh, nbatches = batch_data(dat, batch_size)
        
        # Resample momentum every epoch
        nu = eta_chol @ np.random.normal(size=p).reshape(p,-1) # sample from MV normal
        
        for batch in range(nbatches):
            gradU_batch = gradU(theta, dat_resh[:,:,batch], n, batch_size).reshape(p,-1)
            nu = nu - eta @ gradU_batch - alpha @ nu + \
                 Sig_chol @ np.random.normal(size=p).reshape(p,-1) # sample from MV normal
            theta = theta + nu
            theta_samps[:,it] = theta.reshape(-1,p)
            it = it + 1
        
    return theta_samps

Just try throwing `@jit` in front of things and seeing how much improvement we get.

In [5]:
@jit
def batch_data_numba(data, batch_size):
    n = data.shape[0]
    p = data.shape[1]
    if n % batch_size != 0:
        n = (n // batch_size) * batch_size
    ind = np.arange(n)
    np.random.shuffle(ind)
    n_batches = n // batch_size
    data = data[ind].reshape(batch_size, p, n_batches)
    return(data, n_batches)

In [6]:
@jit
def sghmc_numba(gradU, eta, niter, alpha, theta_0, V_hat, dat, batch_size):
    '''Define SGHMC as described in the paper
    Tianqi Chen, Emily B. Fox, Carlos Guestrin 
    Stochastic Gradient Hamiltonian Monte Carlo 
    ICML 2014.

    The inputs are:
    gradU = gradient of U
    eta = eps^2 M^(-1)
    niter = number of samples to generate
    alpha = eps M^(-1) C
    theta_0 = initial val of parameter(s) to be sampled
    V_hat = estimated covariance matrix of stoch grad noise
    See paper for more details

    The return is:
    A np.array of positions of theta.'''

    ### Initialization and checks ###
    # get dimension of the thing you're sampling
    p = len(theta_0)
    # set up matrix of 0s to hold samples
    n = dat.shape[0]
    theta_samps = np.zeros((p, niter*(n // batch_size)))
    # fix beta_hat as described on pg. 6 of paper
    beta_hat = 0.5 * V_hat @ eta
    # We're sampling from a N(0, 2(alpha - beta_hat) @ eta)
    # so this must be a positive definite matrix
    Sigma = 2 * (alpha - beta_hat) @ eta
    Sig_chol = np.linalg.cholesky(Sigma)
    if not is_pos_def( Sigma ): 
        print("Error: (alpha - beta_hat) eta not pos def")
        return
    # Need batch size to be <= the amount of data
    if (batch_size > n): 
        print("Error: batch_size must be <= number of data points")
        return

    # initialize nu and theta 
    nu = np.random.multivariate_normal(np.zeros(p), eta).reshape(p,-1)
    theta = theta_0
    
    # set up for Chol decomp for MV normal sampling of nu every epoch
    eta_chol = np.linalg.cholesky(eta)
    
    # loop through algorithm to get niter samples
    it = 0
    for i in range(niter):
        dat_resh, nbatches = batch_data_numba(dat, batch_size)
        
        # Resample momentum every epoch
        nu = eta_chol @ np.random.normal(size=p).reshape(p,-1) # sample from MV normal
        
        for batch in range(nbatches):
            gradU_batch = gradU(theta, dat_resh[:,:,batch], n, batch_size).reshape(p,-1)
            nu = nu - eta @ gradU_batch - alpha @ nu + \
                 Sig_chol @ np.random.normal(size=p).reshape(p,-1) # sample from MV normal
            theta = theta + nu
            theta_samps[:,it] = theta.reshape(-1,p)
            it = it + 1
        
    return theta_samps

Maybe the batch_data_numba version won't actually help things... It may just add overhead.

In [7]:
@jit
def sghmc_numba2(gradU, eta, niter, alpha, theta_0, V_hat, dat, batch_size):
    '''Define SGHMC as described in the paper
    Tianqi Chen, Emily B. Fox, Carlos Guestrin 
    Stochastic Gradient Hamiltonian Monte Carlo 
    ICML 2014.

    The inputs are:
    gradU = gradient of U
    eta = eps^2 M^(-1)
    niter = number of samples to generate
    alpha = eps M^(-1) C
    theta_0 = initial val of parameter(s) to be sampled
    V_hat = estimated covariance matrix of stoch grad noise
    See paper for more details

    The return is:
    A np.array of positions of theta.'''

    ### Initialization and checks ###
    # get dimension of the thing you're sampling
    p = len(theta_0)
    # set up matrix of 0s to hold samples
    n = dat.shape[0]
    theta_samps = np.zeros((p, niter*(n // batch_size)))
    # fix beta_hat as described on pg. 6 of paper
    beta_hat = 0.5 * V_hat @ eta
    # We're sampling from a N(0, 2(alpha - beta_hat) @ eta)
    # so this must be a positive definite matrix
    Sigma = 2 * (alpha - beta_hat) @ eta
    Sig_chol = np.linalg.cholesky(Sigma)
    if not is_pos_def( Sigma ): 
        print("Error: (alpha - beta_hat) eta not pos def")
        return
    # Need batch size to be <= the amount of data
    if (batch_size > n): 
        print("Error: batch_size must be <= number of data points")
        return

    # initialize nu and theta 
    nu = np.random.multivariate_normal(np.zeros(p), eta).reshape(p,-1)
    theta = theta_0
    
    # set up for Chol decomp for MV normal sampling of nu every epoch
    eta_chol = np.linalg.cholesky(eta)
    
    # loop through algorithm to get niter samples
    it = 0
    for i in range(niter):
        dat_resh, nbatches = batch_data(dat, batch_size) # use original batch_data
        
        # Resample momentum every epoch
        nu = eta_chol @ np.random.normal(size=p).reshape(p,-1) # sample from MV normal
        
        for batch in range(nbatches):
            gradU_batch = gradU(theta, dat_resh[:,:,batch], n, batch_size).reshape(p,-1)
            nu = nu - eta @ gradU_batch - alpha @ nu + \
                 Sig_chol @ np.random.normal(size=p).reshape(p,-1) # sample from MV normal
            theta = theta + nu
            theta_samps[:,it] = theta.reshape(-1,p)
            it = it + 1
        
    return theta_samps

Look at the times for the above methods

In [8]:
%timeit sghmc(gradU, eta, niter, alpha, theta_0, V, x, batch_size)

1.7 s ± 20.7 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [9]:
%timeit sghmc_cleaned(gradU, eta, niter, alpha, theta_0, V, x, batch_size)

1.56 s ± 25.5 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [10]:
%timeit sghmc_numba(gradU, eta, niter, alpha, theta_0, V, x, batch_size) # use batch_data_numba

1.54 s ± 33.8 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [11]:
%timeit sghmc_numba2(gradU, eta, niter, alpha, theta_0, V, x, batch_size) # use batch_data

1.55 s ± 13 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


Next step should be to do something more fancy, like maybe convert things to C++/Cython. You can't really parallelize the outer epoch loop or the batch function loop, because there is dependency in the Markov chain.

Below, I write the C++ code and test it using Fig 1 (the toy example from the paper). Note that the way I have this written, the gradient function has to also be written in C++, which is a pain because we can't use the auto-gradient like we have in Python.

In [80]:
'''NOTE: Unused things that I didn't end up keeping but want to save just in case are in this block.

// checks whether a matrix X is pos definite, returns bool type
bool not_is_pos_def(Eigen::MatrixXd xs){
    bool res = false; // assume xs is not pos definite
    Eigen::LLT<Eigen::MatrixXd> lltOfX(xs); // compute the Cholesky decomposition of xs
    if(lltOfX.info() != Eigen::NumericalIssue){res = true;} // check if we did not get error  
    return res;
}

// get gradient for mixture of normals example
Eigen::MatrixXd gradU_mixNormals(Eigen::MatrixXd theta, Eigen::MatrixXd x, int n, int batch_size) {
    int p = theta.rows();
    Eigen::MatrixXd star = Eigen::MatrixXd::Zero(n, 1);
    Eigen::MatrixXd star_prime0 = Eigen::MatrixXd::Zero(n, 1);
    Eigen::MatrixXd star_prime1 = Eigen::MatrixXd::Zero(n, 1);
    Eigen::MatrixXd grad = Eigen::MatrixXd::Zero(p, 1);
    for (int i=0; i<n; i++) {
        star(i,0) = 0.5*(-0.5*(theta(0,0)-x(i,0)).pow(2)).exp() + 0.5*(-0.5*(theta(1,0)-x(i,0)).pow(2)).exp();
        star_prime0(i,0) = 0.5*(-0.5*(theta(0,0)-x(i,0)).pow(2)).exp()*(theta(0,0)-x(i,0));
        star_prime1(i,0) = 0.5*(-0.5*(theta(1,0)-x(i,0)).pow(2)).exp()*(theta(1,0)-x(i,0));
    }
    grad(0,0) = -theta(0,0)/10 - (n/batch_size)*(star_prime0.array()/star.array()).sum();
    grad(1,0) = -theta(1,0)/10 - (n/batch_size)*(star_prime1.array()/star.array()).sum();
    return grad;
}
'''
None

In [81]:
%%file wrap.cpp
<%
cfg['compiler_args'] = ['-std=c++11']
cfg['include_dirs'] = ['../notebooks/eigen3']
setup_pybind11(cfg)
%>

#include <pybind11/pybind11.h>
#include <pybind11/eigen.h>
#include <stdexcept>
#include <algorithm> // std::random_shuffle
#include <random>

#include <Eigen/LU>
#include <Eigen/Dense>

namespace py = pybind11;
using std::default_random_engine;
using std::normal_distribution;
        
// start random number engine with fixed seed
default_random_engine re{1234};
// set up random normal rnorm to work like in python
normal_distribution<double> norm(0, 1); // mean and standard deviation
auto rnorm = bind(norm, re);

// fill xs with draws from N(0,1) and return this n x 1 dim vector
Eigen::MatrixXd rnorm_vec(int n) {
    Eigen::MatrixXd xs = Eigen::MatrixXd::Zero(n, 1);
    for (int i=0; i<n; i++) {xs(i,0) = rnorm();}
    return xs;
}
    
// get noisy gradient of Fig1 example from Chen et. al. paper
Eigen::MatrixXd gradU_noisyFig1(Eigen::MatrixXd theta) {
    Eigen::MatrixXd xs = -4*theta.array() + 4*theta.array().pow(3) + 2*rnorm_vec(theta.rows()).array();
    return xs;
} 
    
// Broken gradient calculation
// get gradient for mixture of normals example
Eigen::MatrixXd gradU_mixNormals(Eigen::MatrixXd theta, Eigen::MatrixXd x, int n, int batch_size) {
    int p = theta.rows();
    Eigen::ArrayXd c_0 = theta(0,0) - x.array();
    Eigen::ArrayXd c_1 = theta(1,0) - x.array();
    Eigen::ArrayXd star = 0.5 * (-0.5 * c_0.pow(2)).exp() + 0.5 * (-0.5 * c_1.pow(2)).exp();
    Eigen::ArrayXd star_prime;
    Eigen::MatrixXd grad = Eigen::MatrixXd::Zero(p, 1);
    for (int i=0; i<p; i++) {
        star_prime = 0.5 * (-0.5 * (theta(i,0) - x.array()).pow(2)).exp() * (theta(i,0) - x.array());
        grad(i,0) = -theta(i,0)/10 - (n/batch_size)*(star_prime/star).sum();
    }
    return grad;
} 
    
/*
'''
def gradU_test(theta, x, n, batch_size):
    grad = np.zeros((2,1))
    for p in range(2):
        star = 0.5 * np.exp(-0.5*(theta[0]-x)**2) + 0.5* np.exp(-0.5*(theta[1]-x)**2)
        star_prime = 0.5 * np.exp(-0.5*(theta[p]-x)**2) * (theta[p]-x)
        grad[p,0] = -theta[p]/10 - (n/batch_size)*np.sum(star_prime/star)
    return grad
    
// get correlation between two VectorXd type objects of same length
double corr(Ref<VectorXd const> const& x, Ref<VectorXd const> const& y) {
    ArrayXd c_x = x.array() - x.mean(); // centered x
    ArrayXd c_y = y.array() - y.mean(); // centered y   
    return (c_x * c_y).sum() / sqrt(((c_x*c_x).sum())*((c_y*c_y).sum()));
}
'''
*/

/* 
SGHMC algorithm as described in the paper by Chen et al.
Note that users specify the choice of which gradient function to use by gradU_choice, 
but the gradient function itself must be available in this file
*/
Eigen::MatrixXd sghmc(std::string gradU_choice, Eigen::MatrixXd eta, int niter, Eigen::MatrixXd alpha, Eigen::MatrixXd theta_0, Eigen::MatrixXd V_hat, Eigen::MatrixXd dat, int batch_size){
    // Initialization and checks
    int p = theta_0.rows(); // dimension of the thing you're sampling
    int n = dat.rows();     // number of data observations
    int p_dat = dat.cols(); // 2nd dimension of data
    int nbatches = n / batch_size; // how many batches data will be broken into
    // Set up dat_temp and dat_batch for use in loop below, as well as gradU_batch
    Eigen::MatrixXd dat_temp = dat;
    Eigen::MatrixXd dat_batch = Eigen::MatrixXd::Zero(batch_size, p_dat);
    Eigen::MatrixXd gradU_batch = Eigen::MatrixXd::Zero(p, 1);
    // set up matrix of 0s to hold samples
    Eigen::MatrixXd theta_samps = Eigen::MatrixXd::Zero(p, niter*(n/batch_size));
    // vector to hold indices for shuffling
    std::vector<int> ind;
    // fix beta_hat as described on pg. 6 of paper
    Eigen::MatrixXd beta_hat = 0.5 * V_hat * eta;
    // We're sampling from a N(0, 2(alpha - beta_hat) @ eta)
    // so this must be a positive definite matrix
    Eigen::MatrixXd Sigma = 2.0 * (alpha - beta_hat) * eta;
    Eigen::LLT<Eigen::MatrixXd> lltOfSig(Sigma); // compute the Cholesky decomposition of Sigma
    if(lltOfSig.info() == Eigen::NumericalIssue){ // check if we got error, and break out if so
        return theta_samps; // will just give back all-0s
    }
    Eigen::MatrixXd Sig_chol = lltOfSig.matrixL(); // get L in Chol decomp
    if(batch_size > n){ // Need batch size to be <= the amount of data
        return theta_samps; // will just give back all-0s
    }
    // initialize more things! (nu and theta, to be specific)
    Eigen::LLT<Eigen::MatrixXd> lltOfeta(eta); // compute the Cholesky decomposition of eta
    Eigen::MatrixXd eta_chol = lltOfeta.matrixL(); // get L in Chol decomp
    Eigen::MatrixXd nu = eta_chol * rnorm_vec(p); // initialize nu
    Eigen::MatrixXd theta = theta_0; // initialize theta 
    
    // loop through algorithm to get niter*batch_size samples
    int big_iter = 0;
    for (int it=0; it<niter; it++) {
        
        // shuffle rows of dat to get dat_temp 
        Eigen::VectorXi indices = Eigen::VectorXi::LinSpaced(dat.rows(), 0, dat.rows());
        std::random_shuffle(indices.data(), indices.data() + dat.rows());
        dat_temp = indices.asPermutation() * dat;
        
        // Resample momentum every epoch
        nu = eta_chol * rnorm_vec(p); // sample from MV normal
        
        // loop through the batches
        int count_lower = 0;
        int count_upper = batch_size;
        for (int b=0; b<nbatches; b++){
            int batch_ind = 0;
            for (int ind_temp=count_lower; ind_temp<count_upper; ind_temp++){
                dat_batch.row(batch_ind) = dat_temp.row(ind_temp);
                batch_ind += 1;
            }
            count_lower += batch_size; // add batch size to each iterator
            count_upper += batch_size;
            if (gradU_choice == "fig1"){
                gradU_batch = gradU_noisyFig1(theta);
            } else if (gradU_choice == "mixture_of_normals"){
                gradU_batch = gradU_mixNormals(theta, dat_batch, n, batch_size);
            } else {
                return theta_samps; // will just give back all-0s
            }
            nu = nu - eta * gradU_batch - alpha * nu + Sig_chol * rnorm_vec(p);
            theta = theta + nu;
            theta_samps.col(big_iter) = theta;
            big_iter += 1;
        }
    }

    return theta_samps;
}

PYBIND11_MODULE(wrap, m) {
    m.doc() = "auto-compiled c++ extension";
    m.def("sghmc", &sghmc);
}

Overwriting wrap.cpp


In [13]:
%%file test_code.py
import cppimport
import numpy as np
import seaborn as sns
import time

code = cppimport.imp("wrap")

if __name__ == '__main__':
    np.random.seed(1234)
    # Don't actually need 'data' in this example, just use
    # it as a place-holder to fit into our function.
    n = 100
    x = np.array([np.random.normal(0, 1, (n,1))]).reshape(-1,1)
    # Set up start values and tuning params
    theta_0 = np.array([0.0]) # Initialize theta
    p = theta_0.shape[0]
    eta = 0.001 * np.eye(p) # make this small
    alpha = 0.01 * np.eye(p)
    V = np.eye(p)
    batch_size = n # since we're not actually using the data, don't need to batch it
    niter = 50000 # Not too many iterations, just want to test for speed
    # run SGHMC sampler
    t0 = time.time()
    samps_sghmc = code.sghmc("fig1", eta, niter, alpha, theta_0, V, x, batch_size)
    t1 = time.time()
    
    # how long did it take?
    print("Time taken with C++ version:", t1 - t0, "seconds")
    
    ''' # ADD THIS to save plot to view
    # plot the samples from the algorithm and save to a file
    kdeplt = sns.kdeplot(samps_sghmc.reshape(-1))
    fig = kdeplt.get_figure()
    fig.savefig('Fig1_example.png')
    '''

Overwriting test_code.py


In [14]:
%%bash

python test_code.py

Time taken with C++ version: 0.21611738204956055 seconds


cc1plus: warning: command line option ‘-Wstrict-prototypes’ is valid for C/ObjC but not for C++


In [15]:
import time
import matplotlib.pyplot as plt
import autograd.numpy as np
from autograd import jacobian
import seaborn as sns

### Easy test example based on Figure 1 from the paper
### See pg. 6 of Chen et. al. for their results/comparison

# Log likelihood function
def U(theta):
    return(-2*theta**2 + theta**4)
# True gradient
gradU = jacobian(U, argnum=0)
# Noisy gradient, based on what they do in the paper for Fig 1
def noisy_gradU(theta, x, n, batch_size):
    '''Noisy gradient \Delta\tilde{U}(\theta)=\Delta U(\theta)+N(0,4)
    Extra args (x, n, batch_size) for compatibility with sghmc()'''
    return -4*theta + 4*theta**3 + np.random.normal(0,2)
# Set random seed
np.random.seed(1234)
# Don't actually need 'data' in this example, just use
# it as a place-holder to fit into our function.
n = 100
x = np.array([np.random.normal(0, 1, (n,1))]).reshape(-1,1)
# Set up start values and tuning params
theta_0 = np.array([0.0]) # Initialize theta
p = theta_0.shape[0]
eta = 0.001 * np.eye(p) # make this small
alpha = 0.01 * np.eye(p)
V = np.eye(p)
batch_size = n # since we're not actually using the data, don't need to batch it
niter = 50000 # Lots of iterations

# Below run SGHMC sampler with various implementations
# NOTE may want to do like the %timeit magic does and loop over multiple test runs

In [16]:
# original 
t0 = time.time()
samps_sghmc = sghmc(noisy_gradU, eta, niter, alpha, theta_0, V, x, batch_size)
t1 = time.time()
print("Time taken with original python version:", t1 - t0, "seconds")

Time taken with original python version: 23.926531314849854 seconds


In [17]:
# cleaned
t0 = time.time()
samps_sghmc = sghmc_cleaned(noisy_gradU, eta, niter, alpha, theta_0, V, x, batch_size)
t1 = time.time()
print("Time taken with cleaned up python version:", t1 - t0, "seconds")

Time taken with cleaned up python version: 3.5248875617980957 seconds


In [18]:
# numba (maybe slower because overhead without gains since no data?)
t0 = time.time()
samps_sghmc = sghmc_numba(noisy_gradU, eta, niter, alpha, theta_0, V, x, batch_size)
t1 = time.time()
print("Time taken with Numba version:", t1 - t0, "seconds")

Time taken with Numba version: 6.474595785140991 seconds


Now move on to using the C++ code to test the mixture of normals example, and compare it on time. Note that the way I have this written, the gradient function has to also be written in C++, which is a pain because we can't use the auto-gradient like we have in Python.

In [43]:
import matplotlib.pyplot as plt
import autograd.numpy as np
from autograd import jacobian
import seaborn as sns

## Example #1:
## Sampling from a mixture of normals in 1-D
## SAMPLING MODEL: x ~ 0.5 * N(mu1, 1) + 0.5 * N(mu2, 1)
## PRIORS: p(mu1) = p(mu2) = N(0,10)

def log_prior(theta):
    return(-(1/(2*10))*theta.T@theta)
      
def log_lik(theta, x):
    return(np.log(0.5 * np.exp(-0.5*(theta[0]-x)**2) + 0.5* np.exp(-0.5*(theta[1]-x)**2)))

def U(theta, x, n, batch_size):
    return(-log_prior(theta) - (n/batch_size)*sum(log_lik(theta, x)))
       
# Automatic differentiation to get the gradient
gradU = jacobian(U, argnum=0)

# Set random seed
np.random.seed(1234)
# Set up the data
p = 2 #dimension of theta
theta = np.array([-3.0, 3.0]).reshape(p,-1)
n = 100 # smaller for test
x = np.array([np.random.normal(theta[0], 1, (n,1)),
              np.random.normal(theta[1], 1, (n,1))]).reshape(-1,1)

## Initialize parameters and sample 

# Initialize mean parameters
#theta_0 = np.random.normal(size=(p,1))
theta_0 = theta # initialize at "true" value for testing

# Initialize tuning parameters:
# learning rate
eta = 0.01/n * np.eye(p)
# Friction rate
alpha = 0.1 * np.eye(p)

# Arbitrary guess at covariance of noise from mini-batching the data
V = np.eye(p)*1
niter = 50
batch_size=20 # make this smallish

# TEST MY GRADIENT HAND-CALCULATION 
def gradU_test(theta, x, n, batch_size):
    grad = np.zeros((2,1))
    for p in range(2):
        star = 0.5 * np.exp(-0.5*(theta[0]-x)**2) + 0.5* np.exp(-0.5*(theta[1]-x)**2)
        star_prime = 0.5 * np.exp(-0.5*(theta[p]-x)**2) * (theta[p]-x)
        grad[p,0] = -theta[p]/10 - (n/batch_size)*np.sum(star_prime/star)
    return grad
# THESE SHOULD BE THE SAME!!
print(gradU(theta, x, n, n))
print(gradU_test(theta, x, n, n))

[[[[-3.80991598]
   [ 5.39404395]]]]
[[ 3.80991598]
 [-5.39404395]]


Ok, looks like the gradient works. Now I'll actually check my C++ implementation and make sure it's sampling correctly, then time it. 

In [82]:
%%file test_code_mixture.py
import cppimport
import numpy as np
import seaborn as sns
import time

code = cppimport.imp("wrap")

if __name__ == '__main__':
    np.random.seed(1234)
    ## Example #1:
    ## Sampling from a mixture of normals in 1-D
    ## SAMPLING MODEL: x ~ 0.5 * N(mu1, 1) + 0.5 * N(mu2, 1)
    ## PRIORS: p(mu1) = p(mu2) = N(0,10)

    # Set random seed
    np.random.seed(1234)
    # Set up the data
    p = 2 #dimension of theta
    theta = np.array([-3.0, 3.0]).reshape(p,-1)
    n = 100 # smaller for test
    x = np.array([np.random.normal(theta[0], 1, (n,1)),
                  np.random.normal(theta[1], 1, (n,1))]).reshape(-1,1)

    ## Initialize parameters and sample 

    # Initialize mean parameters
    #theta_0 = np.random.normal(size=(p,1))
    theta_0 = theta # initialize at "true" value for testing

    # Initialize tuning parameters:
    # learning rate
    eta = 0.01/n * np.eye(p)
    # Friction rate
    alpha = 0.1 * np.eye(p)

    # Arbitrary guess at covariance of noise from mini-batching the data
    V = np.eye(p)*1
    niter = 50
    batch_size=20 # make this smallish

    # run SGHMC sampler
    t0 = time.time()
    niter = 1000
    batch_size = n # just for testing
    samps_sghmc = code.sghmc("mixture_of_normals", eta, niter, alpha, theta_0, V, x, batch_size)
    print(samps_sghmc)
    t1 = time.time()
    
    # how long did it take?
    print("Time taken with C++ version:", t1 - t0, "seconds")
    
    '''# ADD THIS to save plot to view
    # plot the samples from the algorithm and save to a file
    kdeplt = sns.kdeplot(samps_sghmc[0,:], samps_sghmc[1,:]) # Plot the joint density
    fig = kdeplt.get_figure()
    fig.savefig('MixNormals_example.png')
    '''

Overwriting test_code_mixture.py


In [83]:
%%bash

python test_code_mixture.py

[[-2.99080802 -2.99886303 -3.00786867 ...         nan         nan
          nan]
 [ 3.00545214  3.01066057  3.0024139  ...         nan         nan
          nan]]
Time taken with C++ version: 0.022101879119873047 seconds


cc1plus: warning: command line option ‘-Wstrict-prototypes’ is valid for C/ObjC but not for C++
